# Feature importance

In [ ]:
"""Feature importance example."""

import pprint

import more_itertools
import numpy as np
import pandas as pd

from skrough.chaos_score import compute_chaos_score
from skrough.dataprep import prepare_df
from skrough.feature_importance import get_feature_importance
from skrough.measures import entropy, gini_impurity

## Dataset

Let's prepare a sample data set - "Play Golf Dataset".

In [ ]:
df = pd.DataFrame(
    np.array(
        [
            ["sunny", "hot", "high", "weak", "no"],
            ["sunny", "hot", "high", "strong", "no"],
            ["overcast", "hot", "high", "weak", "yes"],
            ["rain", "mild", "high", "weak", "yes"],
            ["rain", "cool", "normal", "weak", "yes"],
            ["rain", "cool", "normal", "strong", "no"],
            ["overcast", "cool", "normal", "strong", "yes"],
            ["sunny", "mild", "high", "weak", "no"],
            ["sunny", "cool", "normal", "weak", "yes"],
            ["rain", "mild", "normal", "weak", "yes"],
            ["sunny", "mild", "normal", "strong", "yes"],
            ["overcast", "mild", "high", "strong", "yes"],
            ["overcast", "hot", "normal", "weak", "yes"],
            ["rain", "mild", "high", "strong", "no"],
        ],
        dtype=object,
    ),
    columns=["Outlook", "Temperature", "Humidity", "Wind", "Play"],
)
TARGET_COLUMN = "Play"
print(df)

## Prepare data

Factorize dataset and obtain the sizes of feature domains.

In [ ]:
x, x_domain_sizes, y, y_domain_size = prepare_df(df, TARGET_COLUMN)
column_names = np.array([col for col in df.columns if col != TARGET_COLUMN])

print("Conditional data:")
print(x)
print()
print("Conditional data feature domain sizes:")
print(x_domain_sizes)
print()
print("Target data:")
print(y)
print()
print("Target data feature domain size:")
print(y_domain_size)

## Measure of disorder in the dataset - chaos score

In the context of the given dataset, a chaos score is a quantity that characterizes
a subset of features and, more or less, presents the disorder of decisions in 
the equivalence classes induced by the subsets of features.

In most cases it is reasonable to assume that the chaos score function is monotonic
with respect to subset relation, i.e., for subsets of features $A \subseteq B$,
the chaos score for $A$ should be less or equal to that for $B$.

Attributes are given by their ordinal numbers.

Let's try two standard approaches, i.e., gini_impurity and entropy.

In [ ]:
for chaos_function in [gini_impurity, entropy]:
    print(chaos_function.__name__)
    for attrs in [[0], [0, 1], [0, 1, 3], [0, 1, 2, 3]]:
        print(
            f"chaos score for attrs {attrs}({column_names[attrs]}) = ",
            compute_chaos_score(
                x,
                x_domain_sizes,
                y,
                y_domain_size,
                attrs,
                chaos_fun=chaos_function,
            ),
        )
    print()
    print()

## Assessing feature importance

We can use the above chaos score functions for assessing the features, i.e.,
we can observe the chaos score change if a given feature is removed.

To follow a more realistic example, we can use an enseble of feature subsets, i.e.,
a family of subsets of all atributes, and not just a single subset of features, 
computing the total or average chaos score change over several possible appearances 
of the attribute in the ensemble elements.

In [ ]:
attr_subset_ensemble = [
    [[0, 2], [0, 3], [0], [2, 3], [1, 2, 3]],
    [[0], [0, 1], [1, 2]],
    [list(elem) for elem in more_itertools.powerset(range(4))],
]
for chaos_function in [gini_impurity, entropy]:
    print(chaos_function.__name__)
    for attr_subset in attr_subset_ensemble:
        print("feature importance for attribute subset ensemble: ")
        pprint.pprint(attr_subset, compact=True)
        print(
            get_feature_importance(
                x,
                x_domain_sizes,
                y,
                y_domain_size,
                column_names,
                attr_subset,
                chaos_fun=chaos_function,
            )
        )
        print()
    print()
    print()